<a href="https://colab.research.google.com/github/Nanashi-bot/eduquery/blob/main/Eduquer_Hackathon_review2_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chat with eBook
### Installation

In [4]:
!pip install langchain sentence-transformers chromadb pypdf unstructured pdf2image

In [2]:
!pip install unstructured['pdf']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 112.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma

from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever

In [4]:
import os
from getpass import getpass

HF_token = getpass()

··········


In [5]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

In [6]:
file_path = "./MicroEco.txt"

In [7]:


from langchain_community.document_loaders import UnstructuredFileLoader

In [8]:
data = UnstructuredFileLoader(file_path)

In [9]:
content = data.load()

In [10]:
print(content)

[Document(page_content="income inequality lorez,gini and quintiles\n\nin this section of the course what we're\n\ntalking about is income inequality in\n\nother words why is it that some people\n\nmake more than others and as a part of\n\nthis topic we're going to be talking\n\nabout poverty what it means how its\n\nmeasured and discrimination as one of\n\nthe reasons why some people make more\n\nthan others the first thing we have to\n\ntalk about before we talk about anything\n\nabout income inequality is that when we\n\ndiscuss things like poverty and income\n\ninequality we're not usually talking\n\nabout individuals and how their incomes\n\nare different we're usually talking\n\nabout households so let's first talk for\n\na minute about what a household is so\n\nwhen we talk about an income\n\ndistribution how some people make more\n\nthan others and we're going to look at\n\none of these in a few different ways in\n\na minute we're talking about household\n\nincome and equality s

In [11]:
len(content[0].page_content)

36454

## Chunking- Text Splitter

In [12]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=0)

## Embedding Model

In [13]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "thenlper/gte-large"
)

In [14]:
vectorstore = Chroma(embedding_function=embeddings)
store = InMemoryStore()

In [15]:
model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                       model_kwargs={"temperature":0.85,"max_new_tokens":512,"max_length":64})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [16]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [17]:
retriever.add_documents(content,ids=None)

In [18]:
query = "what is income inequality give me a detailed answer"

In [19]:
relevant_context = retriever.get_relevant_documents(query)

In [20]:
relevant_context

[Document(page_content="as a lot of very high income people live\n\nin this county and lastly let's look at\n\nincome inequality over time in the\n\nUnited States it's kind of hard to tell\n\nwhat these numbers actually mean the a\n\nlot of people say the income inequality\n\nin the United States is getting worse\n\nand looking at the Gini coefficients\n\nthey are they're getting higher nineteen\n\nseventy point three nine two thousand\n\npoint four six now in 2007 the income\n\ninequality actually went down and then", metadata={'source': './MicroEco.txt'}),
 Document(page_content="it's fair for every single household\n\ntype to be earning the same amount of\n\nmoney because it doesn't make sense\n\nwe're not holding the number of people\n\nin a household constant right so this is\n\nanother thing to keep in mind when we\n\ntalk about income inequality what's the\n\ngoal is it to make incomes perfectly\n\nequal and if so what does that even mean\n\nso let's talk about some ways to meas

In [182]:
relevant_context

[Document(page_content="as a lot of very high income people live\n\nin this county and lastly let's look at\n\nincome inequality over time in the\n\nUnited States it's kind of hard to tell\n\nwhat these numbers actually mean the a\n\nlot of people say the income inequality\n\nin the United States is getting worse\n\nand looking at the Gini coefficients\n\nthey are they're getting higher nineteen\n\nseventy point three nine two thousand\n\npoint four six now in 2007 the income\n\ninequality actually went down and then", metadata={'source': './MicroEco.txt'})]

In [21]:
query = "Who is Rahul?"

In [22]:
relevant_context = retriever.get_relevant_documents(query)

In [23]:
relevant_context

[Document(page_content="bottom four groups the bottom 80% are\n\nall very poor they're very equal but\n\nthen most of the income eighty-four\n\npercent of the total is had by just this\n\ntop twenty percent of elites all right\n\nwell let's calculate this cumulative\n\ndistribution and graph it and let's see\n\nwhat it looks like so we always start a\n\ncumulative distribution with just the\n\npercent in the bottom category here so\n\nit's going to be 4 percent saying that\n\nin this first category the poorest 20%", metadata={'source': './MicroEco.txt'}),
 Document(page_content="we're going to take the bottom 20% of\n\nhouseholds the lowest income 20% and\n\nthen the next 20% so if you look at this\n\ntable here the we might call them the\n\npoorest 20% lower middle 20% the middle\n\n20% the upper-middle 20% and the highest\n\n20% of incomes ok second thing we do\n\nthis is named after a guy named Lorenz\n\nhe came up with the idea of making a\n\ngraph called a Lorenz curve and what we

In [ ]:
!pip install cohere

## Step-1 Retrieval

In [ ]:
# os.environ["COHERE_API_KEY"] = "5uuX8mk9dhf9KHzw7vSDhQdXlV2x92MzELvJ972T"

In [ ]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import CohereRerank

In [ ]:
# from cohere import Client

In [ ]:
# co = Client(api_key = "5uuX8mk9dhf9KHzw7vSDhQdXlV2x92MzELvJ972T")

In [ ]:
# from typing import ForwardRef
# from pydantic import BaseModel

# class CustomCohereRerank(CohereRerank):
  # class Config(BaseModel.Config):
    # arbitrary_types_allowed = True

# CustomCohereRerank.update_forward_refs()

In [ ]:
# compressor = CustomCohereRerank(client=co)

In [ ]:
# compression_retriever = ContextualCompressionRetriever(
    # base_compressor=compressor, base_retriever=retriever
# )

## Step - 2 Augment

In [24]:
from langchain_core.prompts import ChatPromptTemplate

In [25]:
template = """
<|system|>>
You are an Ai education Doubt solver for students named EduQuery
whenever user asks you a question, you greet them first and explain the answer
of the question fairly and explain it thouroughly and in a simple manner which is understandable for
everyone. you can add a bit extra if you want own your own to explain the context and the answer
but do not deviate from the topic of the question.
the user will mostly be a student so explain it simply and in full detail and make the answer organized with number points and right parsing

Try to give answer in points when explaining if its needed
CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [26]:
prompt = ChatPromptTemplate.from_template(template)

## Step-3 Generation

In [27]:
from langchain_core.output_parsers import StrOutputParser

In [28]:
from langchain_core.runnables import RunnablePassthrough

In [29]:
output_parser = StrOutputParser()

In [30]:
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [31]:
query = "Who is Rahul?"

In [33]:
response = chain.invoke(query)

In [34]:
print(response)

Human: 
<|system|>>
You are an Ai education Doubt solver for students named EduQuery
whenever user asks you a question, you greet them first and explain the answer 
of the question fairly and explain it thouroughly and in a simple manner which is understandable for 
everyone. you can add a bit extra if you want own your own to explain the context and the answer
but do not deviate from the topic of the question.
the user will mostly be a student so explain it simply and in full detail and make the answer organized with number points and right parsing 

Try to give answer in points when explaining if its needed
CONTEXT: [Document(page_content="bottom four groups the bottom 80% are\n\nall very poor they're very equal but\n\nthen most of the income eighty-four\n\npercent of the total is had by just this\n\ntop twenty percent of elites all right\n\nwell let's calculate this cumulative\n\ndistribution and graph it and let's see\n\nwhat it looks like so we always start a\n\ncumulative distrib

In [36]:
!pip install -q streamlit

In [37]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.424s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [38]:
%%writefile app.py

import streamlit as st

st.video("https://www.youtube.com/watch?v=0IAPZzGSbME")

question = st.text_input('Input question')
#answer= output1(question)
answer = (chain.invoke(question)).split("<|")
st.write('The answer is: ', answer)

uploaded_file = st.file_uploader('Choose your .pdf file', type="pdf")

Overwriting app.py


In [42]:

question = input("enter the question : ")
answer = (chain.invoke(question)).split("<|")
print(answer[3])

enter the question : explain housing unit to me
assistant|>
Certainly! A housing unit is any living space that provides complete independent living facilities for one or more persons. This can include a house, apartment, mobile home, or a group of rooms or a single room occupied or intended for occupancy as separate living quarters. A housing unit can also include shared living spaces or communal facilities like kitchen and bathroom, but the individual or group residing there still has access to their own living space that is separate from others. In short, it's any residence that provides a household with a comfortable and private living environment.


In [39]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.16.174.148


In [40]:
!streamlit run app.py &>/content/logs.txt &

In [41]:
!npx localtunnel --port 8501

npx: installed 22 in 1.756s
your url is: https://fuzzy-chicken-trade.loca.lt
^C
